In [ ]:
import sys
import os

# Define the absolute working directory
base_path = '/kaggle/working'
project_name = 'Double-Pendulum-Simulation'
project_root = os.path.join(base_path, project_name)

# 1. Clone or Pull
if not os.path.exists(project_root):
    os.chdir(base_path)
    !git clone https://github.com/nthday-jpg/Double-Pendulum-Simulation.git
    print("Repository cloned successfully!")
else:
    os.chdir(project_root)
    !git pull
    print("Repository updated successfully!")

# 2. Correctly set the Python Path
# It is vital to add the absolute path to the project root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

os.chdir(base_path)

# 4. Imports
try:
    import torch
    from models.pinn import PINN
    from training.trainer import Trainer
    from data.dataset import get_dataloader
    from utils.config import Config
    print("All modules imported successfully!")
except ImportError as e:
    print(f"Import failed: {e}")
    print(f"Current sys.path: {sys.path}")

In [ ]:
cfg = Config(
    hidden_dims=[64, 64],  
    input_dim=1,
    output_dim=2,
    residual_type="lagrangian",
    t_max=5, t_min=0,
    epochs=1000,
    early_stopping_patience = 50,
    batch_size = 128,
    batch_size_collocation = 512,
    use_compile=False
)

input_path = "/kaggle/input/double-pendulum"

In [ ]:
from accelerate import notebook_launcher
def train_func(config):  
    model = PINN(cfg)

    optimizer = torch.optim.Adam(model.parameters(), lr=cfg.lr)
    data_loader, colloc_loader, val_loader = get_dataloader(
        data_path=f"{input_path}/trajectory_000.npz",
        parameters_path=f"{input_path}/parameters_000.json",
        config=cfg
    )
    trainer = Trainer(
        model=model,
        config=cfg,
        data_loader=data_loader,
        collocation_loader=colloc_loader,
        val_loader=val_loader,
        optimizer=optimizer
    )

    trainer.train()

In [ ]:
notebook_launcher(train_func, args=(cfg,), num_processes=2)